In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
%matplotlib inline

In [4]:
#from math import sin,cos,sqrt,atan2, radians, degrees, atan

In [5]:
from skitracker_utils import calc_distance, clip, calc_gradient, calc_grad_colour

In [60]:
input_file = 'raw_data_collected/ross_sarah_17-20Mar19.csv'

In [61]:
raw  =pd.read_csv(input_file, header=None, names=['Date','Time','Lat','Lon','GPS_Alt','Pressure','Pres_Alt','Temp','Batt'])

In [62]:
#remove the first line if its text
raw=raw[1:]
raw.shape

(148806, 9)

In [63]:
raw.Date.unique()

array(['02/04/2019 ', '03/02/2019 ', '03/17/2019 ', '03/18/2019 ',
       '03/20/2019 '], dtype=object)

In [64]:
raw = raw[raw.Date=='03/20/2019 ']

In [65]:
raw.shape

(19168, 9)

In [66]:
name = 'sarah'
date = datetime(2019,3,20)
output_file = '{}_{}.pkl'.format(name,date.strftime('%d%b%y'))
output_file

'sarah_20Mar19.pkl'

In [67]:
raw['Lat_Rad']=raw.Lat.apply(lambda x: radians(x))
raw['Lon_Rad']=raw.Lon.apply(lambda x: radians(x))

In [68]:
raw['Lat_Delta']=raw.Lat[:-1] - raw.Lat[1:].values # current value minus next value
raw['Lon_Delta']=raw.Lon[:-1] - raw.Lon[1:].values
raw['Lat_Rad_Delta']=raw.Lat_Rad[:-1] - raw.Lat_Rad[1:].values
raw['Lon_Rad_Delta']=raw.Lon_Rad[:-1] - raw.Lon_Rad[1:].values

In [69]:
level = 1.0e-06

In [70]:
data = raw[(abs(raw.Lat_Delta) > level) | (abs(raw.Lon_Delta) > level)]

In [71]:
1.0*data.shape[0]/raw.shape[0]

0.7171327212020033

In [72]:
data['GPS_Alt_Delta']=data.GPS_Alt[1:]-data.GPS_Alt[:-1].values #next value minus current

C:\Users\jjc\Anaconda3\envs\general\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
data['distance']=data.apply(calc_distance,axis=1)

C:\Users\jjc\Anaconda3\envs\general\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [74]:
#gradient
data['gradient'] = data.apply(calc_gradient,axis=1)

C:\Users\jjc\Anaconda3\envs\general\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [75]:
data['grad_colour']=data.apply(calc_grad_colour,axis=1)

C:\Users\jjc\Anaconda3\envs\general\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
data.to_pickle('processed_data/'+output_file)